In [1]:
import numpy as np
from collections import Counter
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras import optimizers
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

Using TensorFlow backend.


# Reading Corpus

In [2]:
corpus_file = open('corpus.txt', mode='r', encoding="utf8")
corpus = corpus_file.read()
corpus = corpus.lower()
print(len(corpus))

21892


In [3]:
vocab_char = set(corpus)
print(vocab_char)

{':', 'w', ')', '_', '”', 'z', 's', 't', 'k', 'n', 'p', '’', 'b', 'y', '“', '!', ';', ']', 'u', '(', 'q', 'e', 'c', 'm', 'd', '[', 'l', 'a', 'i', 'j', '-', 'x', '\n', '?', 'r', 'f', ',', '.', 'g', 'v', ' ', 'h', 'o'}


# Wordwise

In [4]:
dict_punctuation = {
        '.':' ||Period|| ',
        ',':' ||Comma|| ',
        '"':' ||Quotation_Mark|| ',
        ';':' ||Semicolon|| ',
        '!':' ||Exclamation_Mark|| ',
        '?':' ||Question_Mark|| ',
        '(':' ||Left_Parenthesis|| ',
        ')':' ||Right_Parenthesis|| ',
        '--':' ||Double_Dash|| ',
        '-':' ||Dash|| ',
        '_':' ||Underscore|| ',
        '*':' ||Star|| ',
        '\n':' ||Return|| ',
        '’' :' ||Left_Quote|| ',
        '“' :' ||Right_Quotation|| ',
        '”' :' ||Left_Quotation|| ',
        '‘' :' ||Right_Quote|| '
    }

for key, token in dict_punctuation.items():
    corpus = corpus.replace(key, token)
    
word_corpus = corpus.split(' ')
print(word_corpus[1:15])
print(len(word_corpus))

['||Return||', 'the', 'black', 'cat', '||Period||', '', '||Return||', '', '||Return||', 'by', 'edgar', 'allan', 'poe', '||Period||']
5752


In [5]:
from collections import Counter
word_counts = Counter(word_corpus)
print(word_counts)

Counter({'': 640, '||Return||': 376, '||Comma||': 296, 'the': 270, 'of': 178, '||Period||': 176, 'i': 146, '||Dash||': 125, 'and': 122, 'to': 111, 'my': 110, 'a': 92, 'it': 70, '||Underscore||': 57, 'in': 55, 'had': 53, 'was': 51, 'with': 38, 'this': 37, 'me': 37, 'which': 35, 'at': 34, 'as': 33, 'for': 32, 'that': 31, 'not': 30, 'from': 29, 'by': 28, 'but': 25, 'upon': 24, 'one': 23, 'Quotation||': 22, 'been': 22, 'Mark||': 20, 'no': 19, 'its': 18, '||Exclamation': 17, '||Left': 16, '||Semicolon||': 15, 'have': 15, 'into': 15, 'or': 15, 'cat': 14, 'be': 14, 'more': 14, 'very': 13, 'so': 13, '||Right': 13, 'house': 12, 'than': 12, 'an': 11, 'myself': 11, 'even': 11, 'about': 11, 'wife': 10, 'day': 10, 'would': 9, 'night': 9, 'all': 9, 'when': 9, 'heart': 9, 'most': 9, 'is': 9, 'these': 8, 'came': 8, 'now': 8, 'once': 8, 'animal': 8, 'beast': 8, 'some': 8, 'were': 8, 'am': 8, 'wall': 8, 'own': 8, 'knew': 8, 'made': 8, 'could': 8, 'soul': 7, 'pluto': 7, 'he': 7, 'him': 7, 'who': 7, 'cell

In [23]:
vocab = set(word_corpus)
num_classes = len(vocab)
print(num_classes)

vocab_to_int = {c:i for i,c in enumerate(vocab)}
int_to_vocab = {i:c for i,c in enumerate(vocab)}
print(int_to_vocab.get(vocab_to_int.get('||Period||')))

encoded = [vocab_to_int.get(i) for i in word_corpus]
print(encoded[1:10])
print(len(encoded))

1260
||Period||
[1161, 573, 266, 447, 400, 0, 1161, 0, 1161]
5752


In [24]:
steps = 50

In [25]:
X = []
y = []

for i in range(0, len(encoded) - steps, 1):
    X.append(encoded[i : i + steps])
    y.append(encoded[i + steps])

X = np.reshape(X, (len(X), steps))
X = X/float(num_classes)

X_train = X
y_train = np.eye(num_classes)[y]

print(X_train.shape)
print(y_train.shape)


(5702, 50)
(5702, 1260)


In [26]:
# Counter(y)
# print(X[1,:,:])

In [35]:
# Hyperparams
# samples = 1300
dropout = 1
epochs = 100
batch_size = 128
embed_dim = 512

In [36]:
model = Sequential()
model.add(Embedding(input_dim=num_classes, output_dim=embed_dim, input_length=steps))
model.add(LSTM(512, return_sequences = True))
model.add(Dropout(dropout))
model.add(Flatten())
# adam = optimizers.Adam(lr=0.1)
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='adam')

In [37]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [38]:
# model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs, callbacks=callbacks_list)
model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs)
# 5.58

Epoch 1/100
5702/5702 [==============================] - 86s - loss: 5.8564      

KeyboardInterrupt: 

In [58]:
y_pred = model.predict(np.reshape(X_train[102,:],(1,steps)))
print(y_pred)
y_transformed = np.argmax(y_pred)
print(y_transformed)
print(int_to_vocab[y_transformed])

[[  1.34227455e-01   1.44253527e-05   1.20604573e-05 ...,   1.02025828e-04
    1.26223258e-05   1.27296735e-05]]
0



In [59]:
seed = np.random.randint(len(encoded)-steps)
x_seed = corpus[seed:seed + steps]
x_seed = [vocab_to_int[char] for char in x_seed]
x_seed = np.reshape(x_seed, (1, steps, 1))
x_seed = x_seed/float(num_classes)
print(x_seed)

KeyError: ' '

In [60]:
x = x_seed
out = []
charsize = 500

for i in range(charsize):
    y_pred = model.predict(x)
    y_transformed = np.argmax(y_pred)
    output = int_to_vocab[y_transformed]
    out.append(output)
    x_new = y_transformed/float(num_classes)
    x = np.append(x[:,1:100,:], np.reshape(x_new, (1,1,1)), axis = 1)

print('completed')

TypeError: Error when checking model : data should be a Numpy array, or list/dict of Numpy arrays. Found: e lost no opportunity of procuring those of the mo...

In [14]:
print(len(out))
print(''.join(out))

500
ot tead the sout and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the soutd and the
